In [1]:
import sys
sys.path.append('..')

In [2]:
from src.models.cnn import SimpleCNN

In [3]:
model = SimpleCNN()
model

SimpleCNN(
  (conv_stack): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1))
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 16, kernel_size=(3, 3), stride=(1, 1))
    (5): AdaptiveAvgPool2d(output_size=(3, 3))
    (6): Flatten(start_dim=1, end_dim=-1)
  )
  (fc_stack): Sequential(
    (0): Linear(in_features=144, out_features=120, bias=True)
    (1): ReLU()
    (2): Linear(in_features=120, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=5, bias=True)
  )
)

In [6]:
from pathlib import Path
from PIL import Image
import torch
from torch.utils.data import Dataset
from torchvision import transforms


class SpectrogramDataset(Dataset):
    def __init__(self, root_dir: str):
        self.root_dir = Path(root_dir)import torch
from torch.utils.data import random_split

N = len(dataset)

train_size = int(0.6 * N)
val_size = int(0.2 * N)
test_size = N - train_size - val_size  # avoids rounding bugs


        self.classes = sorted(d.name for d in self.root_dir.iterdir() if d.is_dir())
        self.class_to_idx = {cls: i for i, cls in enumerate(self.classes)}

        self.samples = []
        for cls in self.classes:
            for img_path in (self.root_dir / cls).glob("*.png"):
                self.samples.append((img_path, self.class_to_idx[cls]))

        self.transform = transforms.Compose([
            transforms.Grayscale(num_output_channels=1),
            transforms.ToTensor(),  # keeps 600×600
        ])

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = Image.open(img_path)
        image = self.transform(image)
        return image, label


In [1]:
from torch.utils.data import DataLoader
import os

cwd = Path(os.getcwd()).parent
dataset = SpectrogramDataset(Path(cwd, "data", "final"))
loader = DataLoader(dataset, batch_size=8, shuffle=True)

X, y = next(iter(loader))
print(X.shape, y.shape)

NameError: name 'Path' is not defined

In [8]:
import torch
from torch.utils.data import random_split

N = len(dataset)

train_size = int(0.6 * N)
val_size = int(0.2 * N)
test_size = N - train_size - val_size  # avoids rounding bugs

In [9]:
generator = torch.Generator().manual_seed(42)

train_ds, val_ds, test_ds = random_split(
    dataset,
    [train_size, val_size, test_size],
    generator=generator
)

In [10]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=32, shuffle=False)
test_loader  = DataLoader(test_ds, batch_size=32, shuffle=False)


In [11]:
x, y = next(iter(train_loader))
print(x.shape, y.shape)

torch.Size([32, 1, 600, 600]) torch.Size([32])


### Training

In [12]:
## Training the model
from torch import nn, optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = SimpleCNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [13]:
def train_one_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for x, y in loader:
        x = x.to(device)
        y = y.to(device)

        optimizer.zero_grad()

        logits = model(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * x.size(0)
        correct += (logits.argmax(dim=1) == y).sum().item()
        total += y.size(0)

    return running_loss / total, correct / total

In [14]:
@torch.no_grad()
def validate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    for x, y in loader:
        x = x.to(device)
        y = y.to(device)

        logits = model(x)
        loss = criterion(logits, y)

        running_loss += loss.item() * x.size(0)
        correct += (logits.argmax(dim=1) == y).sum().item()
        total += y.size(0)

    return running_loss / total, correct / total


In [ ]:
num_epochs = 20

for epoch in range(num_epochs):
    train_loss, train_acc = train_one_epoch(
        model, train_loader, criterion, optimizer, device
    )

    val_loss, val_acc = validate(
        model, val_loader, criterion, device
    )

    print(
        f"Epoch {epoch+1:02d} | "
        f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.3f} | "
        f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.3f}"
    )

### Evaluate

In [2]:
@torch.no_grad()
def evaluate(model, loader, device):
    model.eval()
    correct = 0
    total = 0

    all_preds = []
    all_labels = []

    for x, y in loader:
        x = x.to(device)
        y = y.to(device)

        logits = model(x)
        preds = logits.argmax(dim=1)

        correct += (preds == y).sum().item()
        total += y.size(0)

        all_preds.append(preds.cpu())
        all_labels.append(y.cpu())

    accuracy = correct / total
    return accuracy, torch.cat(all_preds), torch.cat(all_labels)


NameError: name 'torch' is not defined

In [3]:
test_acc, y_pred, y_true = evaluate(model, test_loader, device)

print(f"Test Accuracy: {test_acc:.3f}")

NameError: name 'evaluate' is not defined

In [4]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true, y_pred)
print(cm)


NameError: name 'y_true' is not defined

In [5]:
import numpy as np

num_classes = 5
for cls in range(num_classes):
    idx = (y_true == cls)
    acc = (y_pred[idx] == y_true[idx]).float().mean().item()
    print(f"Class {cls} accuracy: {acc:.3f}")


NameError: name 'y_true' is not defined